In [1]:
import numpy as np

In [2]:
# Function to calculate facet area given latitude and longitude

def calculate_area_of_facets(lat_res, lon_res, lunar_radius=1737400):
    """
    Calculates the area of lunar facets given the resolution.

    Parameters:
    - lat_res: latitude resolution in degrees
    - lon_res: longitude resolution in degrees
    - lunar_radius: radius of the Moon (default: 1737400 m)

    Returns:
    - area_of_facets: 2D numpy array with facet areas (m²)
    """
    lat_edges = np.radians(np.arange(0, 90+lat_res, lat_res))
    lon_edges = np.radians(np.arange(-180, 180 + lon_res, lon_res))
    area_of_facets = np.zeros((len(lon_edges)-1, len(lat_edges)-1))

    for i in range(len(lon_edges)-1):
        for j in range(len(lat_edges)-1):
            area_of_facets[i, j] = lunar_radius**2 * abs(
                np.sin(lat_edges[j+1]) - np.sin(lat_edges[j])
            ) * abs(lon_edges[i+1] - lon_edges[i])


    return area_of_facets

In [3]:
def calculate_zenith_angle_array(lat_res, lon_res):
    """
    Vectorized calculation of the zenith angle on each facet of lat, lon.
    - Parameters: 
        - Latitude: a numpy array of latitudes
        - Longutide: a numpy array of longitudes
        - Declination (float): a fixed value for declination
    - Returns: Outputs a 3D array of zenith angles with shape (360, 91)
    """
    latitude_edges = np.radians(np.arange(0, 90 + lat_res, lat_res))
    longitude_edges = np.radians(np.arange(-180, 180 + lon_res, lon_res))
    declination = 90


    # Convert angles from degrees to radians for numpy trigonometric functions
    # phase_rad = np.radians(phase)[:, np.newaxis, np.newaxis]  # shape (360, 1, 1)
    longitude_rad = np.radians(longitude_edges)[:, np.newaxis]  # shape (1, 360, 1)
    latitude_rad = np.radians(latitude_edges)[np.newaxis, :]  # shape (1, 1, 91)
    declination_rad = np.radians(declination)

    # Calculate the zenith angle for all combinations of phase, lon, and lat
    zenith_angle_array = np.arccos(
        np.sin(declination_rad) * np.sin(latitude_rad)
        + np.cos(declination_rad) * np.cos(latitude_rad)
        * np.cos(longitude_rad)
    )

    return zenith_angle_array

In [4]:
def flux_input_func(W_0, zenith_angle_array, albedo):
    """
    Calculate the solar flux input to lunar surface.

    Parameters:
    - W_0 (float): Solar irradiance (W/m²).
    - zenith_angle_array (array): Solar zenith angles in radians.
    - albedo (float): Surface reflectivity (unitless).

    Returns:
    - array: Solar flux input per facet (W/m²).
    """
    W_t = W_0 * np.cos(zenith_angle_array) * (1 - albedo)
    return W_t[:-1,:-1]

In [5]:
def thermal_capacitance(BasCap, area_of_facets, tau, rho):
    """
    Calculate thermal capacitance per regolith layer.

    Parameters:
    - BasCap (float): Specific heat capacity of regolith (J/kg·K).
    - area_of_facets (float or array): Area of each facet (m²).
    - tau (float): Thickness of each regolith layer (m).
    - rho (float): Density of lunar regolith (kg/m³).

    Returns:
    - float or array: Thermal capacitance per layer (J/K).
    """
    return BasCap * rho * area_of_facets * tau

In [6]:
def rate_of_change_intermediate(T_np1, T_n, T_nm1, BasCond, tau):
    """
    Calculate the rate of energy change for intermediate layers.

    Parameters:
    - T_np1 (array): Temperature of the layer below current (K).
    - T_n (array): Temperature of current layer (K).
    - T_nm1 (array): Temperature of the layer above current (K).
    - BasCond (float): Thermal conductivity of regolith (W/m·K).
    - tau (float): Thickness of each layer (m).

    Returns:
    - array: Rate of thermal energy change (W/m²).
    """
    return BasCond * (T_np1 - 2 * T_n + T_nm1) / tau**2

In [7]:
def heat_transfer_model(heat_transfer_array, W_0, zenith_angle_array, albedo, sigma,
                        BasCond, rho, BasCap, area_of_facets, tau, delta_t, model_run_time, T_const):
    """
    Perform the lunar regolith heat transfer simulation.

    Parameters:
    - heat_transfer_array (array): 3D array of initial temperatures [lon, lat, depth] (K).
    - W_0 (float): Solar irradiance (W/m²).
    - zenith_angle_array (array): Array of solar zenith angles per facet (radians).
    - albedo (float): Lunar surface albedo (unitless, 0-1).
    - sigma (float): Stefan-Boltzmann constant (W/m²·K⁴).
    - BasCond (float): Thermal conductivity (W/m·K).
    - rho (float): Regolith density (kg/m³).
    - BasCap (float): Specific heat capacity (J/kg·K).
    - area_of_facets (float or array): Area of each facet (m²).
    - tau (float): Thickness of each soil layer (m).
    - delta_t (float): Time-step duration for model (s).
    - model_run_time (float): Total simulation time (s).
    - T_const (float): Constant temperature at deepest layer boundary (K).

    Returns:
    - array: Updated heat_transfer_array after simulation (K).
    """

    gamma = thermal_capacitance(BasCap, area_of_facets, tau, rho)
    num_steps = int(model_run_time / delta_t)

    for step in range(num_steps):
        # Surface Layer update
        T_0 = heat_transfer_array[:, :, 0]
        T_1 = heat_transfer_array[:, :, 1]

        W_t = flux_input_func(W_0, zenith_angle_array, albedo)

        radiative_loss = sigma * T_0**4
        conductive_flux_surface = BasCond * (T_1 - T_0) / tau
        dQ0_dt = (W_t - radiative_loss - conductive_flux_surface) * area_of_facets

        T_0_new = T_0 + (dQ0_dt * delta_t / gamma)

        # Intermediate layers update
        T_n = heat_transfer_array[:, :, 1:-1]
        T_np1 = heat_transfer_array[:, :, 2:]
        T_nm1 = heat_transfer_array[:, :, :-2]

        dQn_dt = rate_of_change_intermediate(T_np1, T_n, T_nm1, BasCond, tau) * area_of_facets[:, :, np.newaxis]
        T_n_new = T_n + (dQn_dt * delta_t / gamma[:, :, np.newaxis])

        # Bottom layer boundary condition
        T_f_new = np.full_like(heat_transfer_array[:, :, -1], T_const)

        # Update temperature array
        heat_transfer_array[:, :, 0] = T_0_new
        heat_transfer_array[:, :, 1:-1] = T_n_new
        heat_transfer_array[:, :, -1] = T_f_new

    return heat_transfer_array


In [8]:

# Define physical constants and parameters
lunar_radius=1737400     # Lunar Radius (m)
sigma = 5.67e-8          # Stefan-Boltzmann constant (W/m²·K⁴)
W_0 = 1365.0             # Solar irradiance (W/m²)
albedo = 0.15            # Lunar surface albedo (unitless)
BasCond = 0.0093         # Basalt thermal conductivity (W/m·K)
rho = 1500               # Lunar regolith density (kg/m³)
BasCap = 670.0           # Basalt specific heat capacity (J/kg·K)
tau = 0.01               # Thickness of each soil layer (m)
delta_t = 60             # Time-step duration for model (s)
model_run_time = 60    # Total simulation time (s)
T_const = 250            # Constant boundary temperature (K)
lat_res = 1
lon_res = 1

In [9]:
heat_transfer_array = np.full((360, 90, 100), T_const).astype(np.float64)

zenith_angle_array = calculate_zenith_angle_array(lat_res, lon_res)
area_of_facets = calculate_area_of_facets(lat_res, lon_res, lunar_radius=1737400)

In [10]:

heat_transfer_model = heat_transfer_model(heat_transfer_array, W_0, zenith_angle_array, albedo, sigma, BasCond, rho, BasCap, area_of_facets, tau, delta_t, model_run_time, T_const)

In [11]:
heat_transfer_model.shape

(360, 90, 100)

In [13]:
heat_transfer_model[180,0]

array([248.67770522, 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.        ,
       250.        , 250.        , 250.        , 250.  